In [1]:
#hide
from utils import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai2.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG 

pd.options.display.max_rows = 20
pd.options.display.max_columns = 100

In [2]:
path = Path("")
df = pd.read_csv(path/'ml_fer_data.csv')
df_exp = pd.read_csv(path/'ml_fer_farms.csv', encoding = "latin")

In [3]:
df.shape

(403405, 29)

40k rows is a little much for my poor mac, cutting it down for faster iteration

In [4]:
df_short=df[:50000]

In [38]:
dep_var = df.columns[13]

In [39]:
df_short = df_short.dropna(subset=[dep_var])

In [40]:
df_valid=df_short[50000:55000]

In [45]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [46]:
dl = TabularDataLoaders.from_df(df_short, y_names=dep_var,
    cat_names = cat,
    cont_names = cont,
    procs = [Categorify, FillMissing, Normalize], splits=splits
                                )

In [47]:
learn = tabular_learner(dl, metrics=mse
                        , loss_func=MSELossFlat(), valid_idx=df_valid
                       )

In [48]:
learn.fit_one_cycle(4, lr= 0.009120108559727669)

epoch,train_loss,valid_loss,mse,time
0,16.669508,12.575871,12.575871,00:27
1,11.834690,11.773903,11.773903,00:29
2,9.304846,11.929216,11.929216,00:29
3,6.391390,12.667510,12.667510,00:28


In [24]:
splits = RandomSplitter()(range_of(df_short))

In [25]:
procs = [Categorify, FillMissing, Normalize]

In [28]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [29]:
to = TabularPandas(df_short, procs, cat, cont, y_names=dep_var, splits=splits)

In [30]:
len(to.train), len(to.valid)

(31664, 7916)

In [31]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [32]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [33]:
def rf(xs, y, n_estimators=40, max_samples=31664,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [34]:
m = rf(xs, y);

In [35]:
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(2.070548, 3.202993)